In [33]:
# Import the required libraries
import cv2
import os

# Function to draw bounding boxes on the image
def draw_bounding_boxes(image_path, annotation_path, output_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Read the annotation file
    with open(annotation_path, 'r') as f:
        annotations = f.readlines()
    
    # Create a dictionary to store the class counts
    class_counts = {}

    # Define the class order
    class_order = [(15, 'Disabled Parking Spot'), (16, 'Parking Spot'), (4, 'Car')]
    
    # Process each annotation
    for annotation in annotations:
        # Parse the annotation values
        class_id, x_center, y_center, width, height = map(float, annotation.split())
        
        # Calculate the bounding box coordinates
        left = int((x_center - width / 2) * image.shape[1])
        top = int((y_center - height / 2) * image.shape[0])
        right = int((x_center + width / 2) * image.shape[1])
        bottom = int((y_center + height / 2) * image.shape[0])
        
        # Set the color based on the class ID
        if class_id == 4:
            color = (0, 165, 255)  # Orange color
        elif class_id == 15:
            color = (255, 0, 0)  # Blue color
        elif class_id == 16:
            color = (0, 255, 0)  # Green color
        
        # Draw the bounding box rectangle on the image
        cv2.rectangle(image, (left, top), (right, bottom), color, 2)
        
        # Update the class count in the dictionary
        class_counts[class_id] = class_counts.get(class_id, 0) + 1
    
    # Add the class count text to the image
    text_position = (image.shape[1] - 400, 30)  # Top-right corner position
    text = f'Disabled Parking Spot Count: {class_counts.get(15, 0)}'
    cv2.putText(image, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    text_position = (text_position[0], text_position[1] + 30)  # Increment the y-coordinate for next class
    
    text = f'Parking Spot Count: {class_counts.get(16, 0)}'
    cv2.putText(image, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    text_position = (text_position[0], text_position[1] + 30)  # Increment the y-coordinate for next class

    text = f'Car Count: {class_counts.get(4, 0)}'
    cv2.putText(image, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    

    # Save the image with bounding boxes
    output_image_path = os.path.join(output_path, os.path.basename(image_path))
    cv2.imwrite(output_image_path, image)


In [34]:
# Specify the folder path containing the images and annotations
folder_path = '../data/imgs/occupied'

# Specify the output folder path
output_folder = '../result/imgs/occupied_with_boxes'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image and annotation in the folder
for image_file in os.listdir(folder_path):
    if image_file.endswith('.jpg'):  # Assuming image files have the .jpg extension
        # Get the paths for the current image and its corresponding annotation file
        image_path = os.path.join(folder_path, image_file)
        annotation_file = os.path.splitext(image_file)[0] + '.txt'
        annotation_path = os.path.join(folder_path, annotation_file)
        
        # Call the function to draw bounding boxes and save the resulting image
        draw_bounding_boxes(image_path, annotation_path, output_folder)

In [35]:
# Specify the folder path containing the images and annotations
folder_path = '../data/imgs/empty'

# Specify the output folder path
output_folder = '../result/imgs/empty_with_boxes'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image and annotation in the folder
for image_file in os.listdir(folder_path):
    if image_file.endswith('.jpg'):  # Assuming image files have the .jpg extension
        # Get the paths for the current image and its corresponding annotation file
        image_path = os.path.join(folder_path, image_file)
        annotation_file = os.path.splitext(image_file)[0] + '.txt'
        annotation_path = os.path.join(folder_path, annotation_file)
        
        # Call the function to draw bounding boxes and save the resulting image
        draw_bounding_boxes(image_path, annotation_path, output_folder)